<a href="https://colab.research.google.com/github/eduardofssouza/gestao-de-leitos/blob/main/Gest%C3%A3o_Leitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problema de Gestão de Leitos Hospitalares**

A **gestão de leitos hospitalares** é um dos desafios mais críticos no gerenciamento de recursos de saúde, principalmente em hospitais com alta demanda de pacientes. A falta de uma gestão eficiente pode levar a superlotação de unidades, tempos de espera elevados para internação, e subutilização de recursos hospitalares, como leitos de enfermaria e Unidades de Terapia Intensiva (UTI).

Nosso objetivo com o problema de gestão de leitos é otimizar a alocação de pacientes em diferentes tipos de leitos, levando em consideração a gravidade do estado de saúde de cada paciente, o tempo de permanência esperado, e a disponibilidade limitada de leitos. Ao fazer isso, buscamos maximizar a utilização eficiente dos recursos hospitalares, minimizando tempos de espera e evitando a subutilização ou sobrecarga de leitos.

Para resolver esse problema, podemos aplicar **técnicas de otimização** da **Pesquisa Operacional**, utilizando programação matemática para alocar os pacientes de forma eficiente. Aqui, modelamos o problema como um **problema de programação inteira**, onde cada paciente deve ser alocado a um leito específico, e respeitamos restrições como a capacidade de leitos de UTI e enfermaria, a compatibilidade entre o tipo de paciente e o tipo de leito, e o tempo máximo de permanência permitido.

### Formulação Matemática

A seguir está a **formulação matemática** do problema de otimização para gestão de leitos hospitalares, considerando as principais variáveis, função objetivo e restrições.

#### 1. **Conjuntos e Parâmetros**

- $( P )$: Conjunto de pacientes.
- $( L )$: Conjunto de leitos.
- $( g_p)$: Gravidade do paciente $( p )$.
- $( t_p )$: Tempo de permanência esperado do paciente $( p )$.
- $( C_k)$: Capacidade total de leitos de UTI.
- $( C_m)$: Capacidade total de leitos de enfermaria.
- $( t_l)$: Tipo do leito $( l )$, onde $( t_l \in \{UTI, Enfermaria\} $).

## 2. Variáveis de Decisão

A variável binária $( x_{p,l})$ indica se o paciente $( p )$ foi alocado ao leito $( l )$.

- $( x_{p,l})$: 1 se o paciente $( p )$ for alocado ao leito $( l )$, 0 caso contrário.

## 3. Função Objetivo

A função objetivo visa minimizar o tempo ponderado pela gravidade dos pacientes:

$$
\text{Minimizar:} \quad \sum_{p \in P} \sum_{l \in L} g_p \cdot x_{p,l}
$$

## 4. Restrições

### 4.1 Alocação Única

Cada paciente deve ser alocado a exatamente um leito:

$$
\sum_{l \in L} x_{p,l} = 1 \quad \forall p \in P
$$

### 4.2 Leito Único

Cada leito só pode ser ocupado por no máximo um paciente:

$$
\sum_{p \in P} x_{p,l} \leq 1 \quad \forall l \in L
$$

### 4.3 Capacidade de Leitos por Tipo

A soma das alocações de leitos de UTI deve respeitar a capacidade máxima:

$$
\sum_{p \in P} \sum_{l \in L : t_l = UTI} x_{p,l} \leq C_{\text{k}}
$$

A soma das alocações de leitos de enfermaria também deve respeitar a capacidade:

$$
\sum_{p \in P} \sum_{l \in L : \text{tipo}_l = \text{Enfermaria}} x_{p,l} \leq C_{\text{m}}
$$

### 4.4 Compatibilidade Paciente-Leito

Pacientes com gravidade menor que 3 não podem ser alocados a leitos de UTI:

$$
x_{p,l} = 0 \quad \forall p \in P \ \text{onde} \ g_p < 3, \ \forall l \in L \ \text{onde} \ \text{tipo}_l = \text{UTI}
$$

### 4.5 Tempo Máximo de Permanência

Pacientes com tempo de permanência superior a 7 dias não podem ser alocados:

$$
x_{p,l} = 0 \quad \forall p \in P \ \text{onde} \ t_p > 7, \ \forall l \in L
$$


---

### Explicação para o Colab

Agora que temos a formulação matemática do problema, podemos partir para a implementação prática no **Google Colab** usando **Pyomo**. O modelo será rodado com 5 instâncias de dados fictícios diferentes, onde o objetivo é verificar como a alocação de pacientes aos leitos ocorre com base na gravidade e nas restrições definidas.

Seguindo a formulação apresentada, a próxima etapa será rodar o código, que será responsável por gerar instâncias de pacientes e leitos, criar o modelo de otimização e resolver o problema utilizando o solver GLPK.



## **1. Instalando as Dependências**

No **Google Colab**, você precisará instalar o **Pyomo** e o solver **GLPK** para resolver o modelo. Adicione a célula abaixo para instalar essas dependências:

In [1]:
# Instalação do Pyomo e do solver GLPK
!pip install pyomo
!apt-get install -y -qq glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.8 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123599 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package

## **2. Importação de Bibliotecas**

A primeira célula será para importar todas as bibliotecas necessárias, incluindo Pyomo e bibliotecas padrão de Python para manipulação:

In [2]:
# Importar as bibliotecas necessárias
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Binary
import random

## **3. Função para Gerar Instâncias de Dados**

Essa função vai gerar os dados fictícios dos pacientes e leitos, ajustando o número de pacientes e leitos conforme necessário. Adicione a célula abaixo para gerar instâncias viáveis:

In [3]:
# Função para gerar instâncias de pacientes e leitos
def gerar_instancias(num_pacientes=10, num_leitos=10):
    """
    Gera dados fictícios viáveis para pacientes e leitos hospitalares.

    Args:
        num_pacientes (int): Número de pacientes.
        num_leitos (int): Número de leitos.

    Returns:
        dict: Dicionário com instâncias de dados.
    """

    # Gerar tempos de internação e gravidade de pacientes (entre 1 a 10 dias de internação)
    pacientes = {
        f'paciente_{i+1}': {
            'tempo_internacao': random.randint(1, 7),  # tempo de permanência (dias)
            'gravidade': random.randint(1, 5)  # gravidade do paciente
        }
        for i in range(num_pacientes)
    }

    # Garantir que haja um número de leitos razoável para atender à demanda
    assert num_leitos >= num_pacientes // 2, "Número de leitos deve ser pelo menos metade do número de pacientes"

    # Gerar disponibilidade de leitos e tipos de leitos (balanceado entre UTI e Enfermaria)
    leitos = {
        f'leito_{j+1}': {
            'tipo': 'UTI' if j < num_leitos // 2 else 'Enfermaria',  # primeira metade dos leitos é UTI, o restante é enfermaria
            'disponivel': True  # disponibilidade inicial
        }
        for j in range(num_leitos)
    }

    # Capacidade: número máximo de pacientes que podem ser alocados por tipo de leito
    capacidade = {
        'UTI': num_leitos // 2,  # Capacidade máxima para UTI é metade dos leitos totais
        'Enfermaria': num_leitos - num_leitos // 2  # Capacidade para enfermaria é o restante dos leitos
    }

    return {
        'pacientes': pacientes,
        'leitos': leitos,
        'capacidade': capacidade
    }

## **4. Função para Criar o Modelo de Otimização**

Essa célula vai criar o modelo de otimização em Pyomo, incluindo as restrições que definimos:

In [4]:
# Função para criar o modelo de otimização
def criar_modelo(instancias):
    """
    Cria o modelo de otimização para alocação de leitos hospitalares com mais restrições.

    Args:
        instancias (dict): Instâncias de dados de pacientes e leitos.

    Returns:
        model: Modelo Pyomo.
    """
    # Dados
    pacientes = instancias['pacientes']
    leitos = instancias['leitos']
    capacidade = instancias['capacidade']

    # Modelo Pyomo
    model = ConcreteModel()

    # Conjuntos
    model.P = list(pacientes.keys())  # Conjunto de pacientes
    model.L = list(leitos.keys())  # Conjunto de leitos

    # Variáveis: x[p,l] = 1 se o paciente p for alocado ao leito l
    model.x = Var(model.P, model.L, domain=Binary)

    # Função Objetivo: maximizar a gravidade dos pacientes alocados
    def objetivo(model):
        return sum(model.x[p, l] * pacientes[p]['gravidade'] for p in model.P for l in model.L)

    model.objetivo = Objective(rule=objetivo, sense=-1)  # Minimizar tempo de internação

    # Restrição: cada paciente deve ser alocado a um único leito
    def restricao_paciente(model, p):
        return sum(model.x[p, l] for l in model.L) == 1

    model.restricao_paciente = Constraint(model.P, rule=restricao_paciente)

    # Restrição: cada leito só pode receber um paciente
    def restricao_leito(model, l):
        return sum(model.x[p, l] for p in model.P) <= 1

    model.restricao_leito = Constraint(model.L, rule=restricao_leito)

    # Restrição: respeitar a capacidade dos tipos de leito
    def restricao_capacidade_UTI(model):
        return sum(model.x[p, l] for p in model.P for l in model.L if leitos[l]['tipo'] == 'UTI') <= capacidade['UTI']

    def restricao_capacidade_enfermaria(model):
        return sum(model.x[p, l] for p in model.P for l in model.L if leitos[l]['tipo'] == 'Enfermaria') <= capacidade['Enfermaria']

    model.restricao_capacidade_UTI = Constraint(rule=restricao_capacidade_UTI)
    model.restricao_capacidade_enfermaria = Constraint(rule=restricao_capacidade_enfermaria)

    # Restrição: compatibilidade paciente-leito (somente pacientes críticos podem usar UTI)
    def restricao_compatibilidade(model, p, l):
        if leitos[l]['tipo'] == 'UTI' and pacientes[p]['gravidade'] < 3:
            return model.x[p, l] == 0
        return Constraint.Skip

    model.restricao_compatibilidade = Constraint(model.P, model.L, rule=restricao_compatibilidade)

    # Restrição: tempo de permanência máxima de pacientes em leitos (máximo de 7 dias)
    def restricao_tempo_maximo(model, p, l):
        if pacientes[p]['tempo_internacao'] > 7:
            return model.x[p, l] == 0
        return Constraint.Skip

    model.restricao_tempo_maximo = Constraint(model.P, model.L, rule=restricao_tempo_maximo)

    return model


## **5. Executando o Modelo e Rodando Múltiplas Instâncias**

Essa função vai gerar e resolver `n` instâncias diferentes, exibindo os resultados, incluindo o valor da função objetivo e a alocação de pacientes aos leitos:

In [7]:
# Função para rodar o modelo em múltiplas instâncias
def main(num_instancias=5, num_pacientes=10, num_leitos=10):
    solver = SolverFactory('glpk')

    for i in range(num_instancias):
        print(f"\n=== Executando Instância {i+1} ===\n")

        # Gerar instâncias de dados fictícios viáveis
        instancias = gerar_instancias(num_pacientes=num_pacientes, num_leitos=num_leitos)

        # Exibir os dados da instância gerada
        print("Instância Gerada:")
        print("\nPacientes:")
        for p, dados in instancias['pacientes'].items():
            print(f"{p}: Tempo de Permanência = {dados['tempo_internacao']}, Gravidade = {dados['gravidade']}")

        print("\nLeitos:")
        for l, dados in instancias['leitos'].items():
            print(f"{l}: Tipo = {dados['tipo']}, Disponível = {dados['disponivel']}")

        print("\nCapacidade:")
        print(f"UTI: {instancias['capacidade']['UTI']}, Enfermaria: {instancias['capacidade']['Enfermaria']}\n")

        # Criar modelo de otimização
        modelo = criar_modelo(instancias)

        try:
            # Resolver o modelo
            resultado = solver.solve(modelo)

            # Verificar se o solver encontrou uma solução viável
            if (resultado.solver.status == 'ok') and (resultado.solver.termination_condition == 'optimal'):
                # Imprimir o valor da função objetivo
                valor_objetivo = modelo.objetivo()
                print(f"\nValor da Função Objetivo: {valor_objetivo:.2f}\n")

                # Exibir a alocação de pacientes aos leitos
                print("\nAlocação de Pacientes aos Leitos:\n")
                for p in modelo.P:
                    for l in modelo.L:
                        if modelo.x[p, l].value == 1:
                            print(f"{p} foi alocado ao {l} ({instancias['leitos'][l]['tipo']})")
            else:
                print("\nInstância inviável: O solver não encontrou uma solução ótima.\n")
        except ValueError as e:
            # Tratamento de erro se a variável de decisão não tiver valor
            print(f"Erro na resolução da instância {i+1}: {str(e)}")
            print("Instância inviável, passando para a próxima...\n")

if __name__ == "__main__":
    main()


=== Executando Instância 1 ===

Instância Gerada:

Pacientes:
paciente_1: Tempo de Permanência = 4, Gravidade = 1
paciente_2: Tempo de Permanência = 2, Gravidade = 5
paciente_3: Tempo de Permanência = 1, Gravidade = 5
paciente_4: Tempo de Permanência = 3, Gravidade = 1
paciente_5: Tempo de Permanência = 1, Gravidade = 2
paciente_6: Tempo de Permanência = 7, Gravidade = 4
paciente_7: Tempo de Permanência = 6, Gravidade = 3
paciente_8: Tempo de Permanência = 6, Gravidade = 3
paciente_9: Tempo de Permanência = 4, Gravidade = 1
paciente_10: Tempo de Permanência = 7, Gravidade = 2

Leitos:
leito_1: Tipo = UTI, Disponível = True
leito_2: Tipo = UTI, Disponível = True
leito_3: Tipo = UTI, Disponível = True
leito_4: Tipo = UTI, Disponível = True
leito_5: Tipo = UTI, Disponível = True
leito_6: Tipo = Enfermaria, Disponível = True
leito_7: Tipo = Enfermaria, Disponível = True
leito_8: Tipo = Enfermaria, Disponível = True
leito_9: Tipo = Enfermaria, Disponível = True
leito_10: Tipo = Enfermaria,